In [27]:
#pip install autogen openai

#### Note

By default, Microsoft AutoGen assumes agents may execute code, so it tries to run that code inside Docker for safety.

Set Docker as False

In [12]:
from autogen import AssistantAgent, UserProxyAgent

In [13]:
user_profile = """
John has 6 years of experience as a software engineer.
He has worked on backend systems using Python, Django, and Flask.
He has experience with cloud platforms like AWS and Docker.
He occasionally mentors junior developers.
"""

In [14]:

# 1. LLM Configuration
llm_config = {
    "model": "gpt-4o-mini",   # can change to gpt-4, gpt-3.5-turbo etc.
    "temperature": 0.2
}

# 2. Assistant Agent (AI Brain)
assistant = AssistantAgent(
    name="AI_Assistant",
    llm_config=llm_config,
    system_message="""
    You are an AI assistant helping professionals understand Agentic AI.
    Analyze user Resume based on Experience, Skills, and Role
    """
)

# 3. User Proxy Agent (Human Controller)
user_proxy = UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=3,
    code_execution_config={
        "use_docker": False
    }
)

# 4. Start the Conversation
user_proxy.initiate_chat(
    assistant,
    message=user_profile
)


User (to AI_Assistant):


John has 6 years of experience as a software engineer.
He has worked on backend systems using Python, Django, and Flask.
He has experience with cloud platforms like AWS and Docker.
He occasionally mentors junior developers.


--------------------------------------------------------------------------------
AI_Assistant (to User):

Based on the provided information, here’s an analysis of John’s resume focusing on his experience, skills, and role:

### Experience:
- **Years of Experience**: John has 6 years of experience as a software engineer, which indicates a solid foundation in the field.
- **Backend Development**: His focus on backend systems suggests he has a deep understanding of server-side logic, database management, and API development.

### Skills:
- **Programming Languages**: Proficient in Python, which is a highly sought-after language for backend development.
- **Frameworks**: Experience with Django and Flask indicates he is well-versed in web frame

ChatResult(chat_id=76804606678820682085214154903758403415, chat_history=[{'content': '\nJohn has 6 years of experience as a software engineer.\nHe has worked on backend systems using Python, Django, and Flask.\nHe has experience with cloud platforms like AWS and Docker.\nHe occasionally mentors junior developers.\n', 'role': 'assistant', 'name': 'User'}, {'content': 'Based on the provided information, here’s an analysis of John’s resume focusing on his experience, skills, and role:\n\n### Experience:\n- **Years of Experience**: John has 6 years of experience as a software engineer, which indicates a solid foundation in the field.\n- **Backend Development**: His focus on backend systems suggests he has a deep understanding of server-side logic, database management, and API development.\n\n### Skills:\n- **Programming Languages**: Proficient in Python, which is a highly sought-after language for backend development.\n- **Frameworks**: Experience with Django and Flask indicates he is well

            Step-by-step flow:

            UserProxyAgent sends a task

            AssistantAgent reasons using LLM

            Response flows back

            Conversation can continue automatically

In [20]:
skill_agent = AssistantAgent(
    name="SkillAgent",
    llm_config={"model": "gpt-4o-mini"},
    system_message="Evaluate technical skills."
)

experience_agent = AssistantAgent(
    name="ExperienceAgent",
    llm_config={"model": "gpt-4o-mini"},
    system_message="Evaluate experience and seniority."
)


In [21]:
from autogen import GroupChat, GroupChatManager

group_chat = GroupChat(
    agents=[skill_agent, experience_agent],
    messages=[],
    speaker_selection_method="round_robin"
)

manager = GroupChatManager(
    groupchat=group_chat,
    llm_config={"model": "gpt-4o-mini"}
)


In [22]:
user = UserProxyAgent(
    name="user",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=3,
    code_execution_config={"use_docker": False},
    is_termination_msg=lambda msg: "FINAL ANSWER" in msg["content"]
)

In [23]:
user.initiate_chat(
    manager,
    message="Evaluate this resume: 6 years Python backend engineer."
)


user (to chat_manager):

Evaluate this resume: 6 years Python backend engineer.

--------------------------------------------------------------------------------

Next speaker: SkillAgent

SkillAgent (to chat_manager):

To evaluate a resume for a Python backend engineer with 6 years of experience, consider the following key areas:

1. **Technical Skills:**
   - Proficiency in Python: Look for specific frameworks (e.g., Django, Flask, FastAPI) the candidate has worked with.
   - Database Management: Experience with SQL (PostgreSQL, MySQL) and NoSQL databases (MongoDB, Redis) should be noted.
   - API Development: Familiarity with RESTful services and GraphQL APIs.
   - Familiarity with cloud services: Experience with AWS, Azure, or Google Cloud Platform.
   - Understanding of version control systems, preferably Git.
   - Knowledge of software development methodologies (Agile/Scrum).

2. **Project Experience:**
   - Significant projects that demonstrate problem-solving abilities using Py

ChatResult(chat_id=111703472482410096413779434782523514724, chat_history=[{'content': 'Evaluate this resume: 6 years Python backend engineer.', 'role': 'assistant', 'name': 'user'}], summary='Evaluate this resume: 6 years Python backend engineer.', cost={'usage_including_cached_inference': {'total_cost': 0}, 'usage_excluding_cached_inference': {'total_cost': 0}}, human_input=[])

### AutoGen needs a Agent which acts as final Agent and gives response. User has to define this agent clearly

In [26]:
# ===============================
# AutoGen: Single-Cell Demo
# Resume Evaluation with FINAL ANSWER
# ===============================

from autogen import (
    AssistantAgent,
    UserProxyAgent,
    GroupChat,
    GroupChatManager
)

# -------- 1. LLM Config --------
llm_config = {
    "model": "gpt-4o-mini",
    "temperature": 0
}

# -------- 2. Define Agents --------
skill_agent = AssistantAgent(
    name="SkillAgent",
    llm_config=llm_config,
    system_message="""
    You evaluate technical skills from a resume.
    Be concise and factual.
    """
)

experience_agent = AssistantAgent(
    name="ExperienceAgent",
    llm_config=llm_config,
    system_message="""
    You evaluate experience and seniority level.
    Focus on years and role maturity.
    """
)

reviewer_agent = AssistantAgent(
    name="ReviewerAgent",
    llm_config=llm_config,
    system_message="""
    You are the final reviewer.
    Summarize inputs from other agents.
    Provide a clear decision.
    End your response with:
    FINAL ANSWER:
    """
)

# -------- 3. Group Chat Setup --------
# group_chat = GroupChat(
#     agents=[skill_agent, experience_agent, reviewer_agent],
#     messages=[],
#     speaker_selection_method="round_robin",
#     allow_repeat_speaker=False,
#     max_round=5
# )
group_chat = GroupChat(
    agents=[skill_agent, experience_agent, reviewer_agent],
    messages=[],
    speaker_selection_method="round_robin",
    allow_repeat_speaker=False,
    max_round=3
)

manager = GroupChatManager(
    groupchat=group_chat,
    llm_config=llm_config
)

# -------- 4. User Proxy with Termination --------
user = UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
    code_execution_config={"use_docker": False},
    is_termination_msg=lambda msg: "FINAL ANSWER" in msg["content"]
)

# -------- 5. Run the Chat --------
chat_result = user.initiate_chat(
    manager,
    message="Evaluate this resume: 6 years Python backend engineer."
)

# -------- 6. Extract FINAL RESPONSE (LangChain-style) --------
final_response = [
    msg["content"]
    for msg in chat_result.chat_history
    if msg["name"] == "ReviewerAgent"
][-1]

print("====== FINAL RESPONSE ======\n")
print(final_response)


User (to chat_manager):

Evaluate this resume: 6 years Python backend engineer.

--------------------------------------------------------------------------------

Next speaker: SkillAgent

SkillAgent (to chat_manager):

The candidate has 6 years of experience as a Python backend engineer, indicating a solid foundation in backend development using Python. This level of experience suggests proficiency in relevant frameworks, libraries, and tools commonly used in Python backend development, such as Django or Flask, as well as familiarity with database management, API development, and possibly cloud services. Further details on specific projects, technologies used, and contributions would provide a clearer picture of their technical skills and expertise.

--------------------------------------------------------------------------------

Next speaker: ExperienceAgent

ExperienceAgent (to chat_manager):

With 6 years of experience as a Python backend engineer, the candidate is positioned at a

IndexError: list index out of range